In [1]:
import os
import torch
import requests

os.environ['CUDA_VISIBLE_DEVICES'] = '3'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [3]:
torch.cuda.empty_cache()

import gc
gc.collect()

0

In [4]:
from transformers import InstructBlipProcessor, InstructBlipForConditionalGeneration
import torch
from PIL import Image

In [5]:
model = InstructBlipForConditionalGeneration.from_pretrained("Salesforce/instructblip-vicuna-7b", cache_dir="/home/raid/hf_cache").to(device)
processor = InstructBlipProcessor.from_pretrained("Salesforce/instructblip-vicuna-7b", cache_dir="/home/raid/hf_cache")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [15]:
# url = "https://raw.githubusercontent.com/salesforce/LAVIS/main/docs/_static/Confusing-Pictures.jpg"
image = Image.open("/home/raid/datasets/hack_museum/train/83137/20110231.jpg").convert("RGB")

In [33]:
prompt = f"""Your task is to beautifully describe the work of art, what is in the image.
Keep in mind that the author of this work is {}
The work is called: "{}"
The work was written on this date: "{}"

Describe it according to the template:
#Author, #title, #date of creation
#your generated description#"""
inputs = processor(images=image, text=prompt, return_tensors="pt").to(device)

In [34]:
outputs = model.generate(
    **inputs,
    do_sample=False,
    num_beams=5,
    max_length=256,
    min_length=100,
    top_p=0.9,
    repetition_penalty=1.5,
    length_penalty=1.0,
    temperature=1,
)
generated_text = processor.batch_decode(outputs, skip_special_tokens=True)[0].strip()
print(generated_text)

"Картина маслом" (Painting in oil) - 1826. The painting depicts a vase with flowers and fruits on a table. The author of the painting is not known. The date of creation is 1826. The size of the painting is 50 x 70 cm. The technique of the painting is oil on canvas. The style of the painting is Realism. The subject of the painting is still life.


In [35]:
generated_text

'"Картина маслом" (Painting in oil) - 1826. The painting depicts a vase with flowers and fruits on a table. The author of the painting is not known. The date of creation is 1826. The size of the painting is 50 x 70 cm. The technique of the painting is oil on canvas. The style of the painting is Realism. The subject of the painting is still life.'

In [9]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

translate_model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_1.2B", cache_dir="/home/raid/hf_cache").to(device)
translate_tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_1.2B", src_lang='en', tgt_lang='ru', cache_dir="/home/raid/hf_cache")

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


tokenizer_config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

In [36]:
translate_inputs = translate_tokenizer(generated_text, return_tensors='pt').to(device)
generated_tokens = translate_model.generate(**translate_inputs, forced_bos_token_id=translate_tokenizer.get_lang_id("ru"))
translate_tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

['"Картина маслом" (живопись в масле) - 1826 год. Картина изображает вазу с цветами и фруктами на столе. Автор картины неизвестен. Дата создания - 1826. Размер картины 50 x 70 см. Техника живописи - масло на холсте. Стиль картины - реализм. Предмет картины - еще жизнь.']